# Import Libraries

In [3]:
import requests
import numpy as np
import pandas as pd
import json

from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


# Scrap data From Wikipedia Page

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [4]:
from bs4 import BeautifulSoup

In [5]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page, 'lxml')

In [18]:
dict_toronto = {'PostalCode':[],
                'Borough':[],
               'Neighborhood':[]}

tbody = soup.find('tbody')
for tr in tbody.find_all('tr')[1:]:
    td = tr.find_all('td')
    pocode = td[0].text
    if td[2].a:
        neigh = td[2].a.text
        if td[1].a:
            boro = td[1].a.text
        else:
            boro=td[1].text
    else:
        neigh = (' ').join(td[2].text.split())
        if td[1].a:
            boro = td[1].a.text
        else:
            boro=td[1].text
    
    if neigh == 'Not assigned':
        neigh = boro
    
    if len(dict_toronto['PostalCode'])==0:
        dict_toronto['PostalCode'].append(pocode)
        dict_toronto['Neighborhood'].append(neigh)
        dict_toronto['Borough'].append(boro)
    else:
        if dict_toronto['PostalCode'][-1] == pocode:
            dict_toronto['Neighborhood'][-1] = dict_toronto['Neighborhood'][-1]+', ' + neigh
        else:
            dict_toronto['PostalCode'].append(pocode)
            dict_toronto['Neighborhood'].append(neigh)
            dict_toronto['Borough'].append(boro)

        

In [20]:
toronto_df = pd.DataFrame.from_dict(dict_toronto)

In [23]:
toronto_df.replace('Not assigned', np.nan, inplace=True)
toronto_df.dropna(inplace=True)
toronto_df.reset_index(drop=True, inplace=True)
toronto_df

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Queen's Park,Queen's Park,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Rouge, Malvern",M1B
7,North York,Don Mills North,M3B
8,East York,"Woodbine Gardens, Parkview Hill",M4B
9,Downtown Toronto,"Ryerson, Garden District",M5B


In [24]:
toronto_df.to_csv('Toronto_neighborhood.csv')

In [25]:
toronto_df.shape

(103, 3)